In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names-shuffled.txt', 'r').read().splitlines()

In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {c:i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {v:k for k, v in stoi.items()}
vocab_size = len(itos)
print(vocab_size)

27


In [4]:
context_length = 3 # context length: number of chars used to predict the next char

def build_dataset(words):
    X, Y = [], [] # inputs and labels

    for w in words:
        context = [0] * context_length
        for c in w + '.':
            X.append(context)
            Y.append(stoi[c])
            context = context[1:] + [stoi[c]] # sliding the context by 1 character to right

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])      # 80%
Xdev, Ydev = build_dataset(words[n1:n2])  # 10%
Xte, Yte = build_dataset(words[n2:])      # 10%

torch.Size([404420, 3]) torch.Size([404420])
torch.Size([50599, 3]) torch.Size([50599])
torch.Size([50438, 3]) torch.Size([50438])


In [5]:
# to compare manual gradients with PyTorch gradients

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item() # exactly equal
    app = torch.allclose(dt, t.grad)    # approx equal
    maxdiff = (dt - t.grad).abs().max().item() # maximum difference
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [12]:
n_emb_dim = 10    # embedding dimensionality
n_hidden = 64    # neurons in hidden layers

g = torch.Generator().manual_seed(1337101)

C = torch.randn((vocab_size, n_emb_dim), generator=g)
# Layer 1
W1 = torch.randn((context_length*n_emb_dim, n_hidden), generator=g) * (5/3)/((context_length*n_emb_dim)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 # useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# batch normalization params
bn_gain = torch.ones((1, n_hidden))*0.1 + 1.0
bn_bias = torch.zeros((1, n_hidden))*0.1

# grads not required for these
# bn_mean_running = torch.zeros((1, n_hidden))
# bn_std_running = torch.ones((1, n_hidden))

# added bn_gain and bn_bias
parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]

for p in parameters:
    p.requires_grad = True

print(f'Total Parameters: {sum(p.nelement() for p in parameters)}')

Total Parameters: 4137


In [13]:
batch_size = 24
n = batch_size # variable for convenience

# creating a mini-batch
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [14]:
# forward pass

emb = C[Xb]
embcat = emb.view(-1, W1.shape[0])
# Linear layer 1
hprebn = embcat@W1 + b1 # hidden layer pre-BN
# BN layer
bnmeani = (1/n)*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = (1/(n-1))*(bndiff2).sum(0, keepdim=True) # Bessel's Correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**(-0.5) # 1/sqrt(bnvar + 1e-5)
bnraw = bndiff*bnvar_inv
hpreact = bn_gain * bnraw + bn_bias # hidden layer pre-activation
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h@W2 + b2 # output layer
# cross entropy loss (F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numeric stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean() # indexing the correct (Yth) char prob from each 24 (n) rows

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss


tensor(3.4895, grad_fn=<NegBackward0>)

In [15]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n

dprobs = dlogprobs * (1.0/probs)

dcounts_sum_inv = (dprobs * counts).sum(1, keepdim=True) # when a node is replicated then its gradient is added during backpropagation

dcounts_sum = dcounts_sum_inv * (-counts_sum**-2)

dcounts = (dprobs * counts_sum_inv) + (dcounts_sum * torch.ones_like(counts))

dnorm_logits = dcounts * norm_logits.exp()

dlogit_maxes = (dnorm_logits * (-torch.ones_like(logits))).sum(1, keepdim=True)

dlogits = (dnorm_logits * torch.ones_like(logits)) + (dlogit_maxes * F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]))

dh = dlogits @ W2.T

dW2 = h.T @ dlogits

db2 = (dlogits * torch.ones_like(h@W2)).sum(dim=0)

dhpreact = dh * (1 - torch.tanh(hpreact)**2)

dbngain = (dhpreact * bnraw).sum(dim=0, keepdim=True)

dbnbias = (dhpreact * torch.ones_like(bn_bias)).sum(dim=0, keepdim=True)

dbnraw = dhpreact * bn_gain

dbnvar_inv = (dbnraw * bndiff).sum(dim=0, keepdim=True)

dbnvar = dbnvar_inv * (-0.5 * (bnvar+ 1e-5) ** (-1.5))

dbndiff2 = dbnvar * (1/(n-1))*torch.ones_like(bnvar)

dbndiff = (dbnraw * bnvar_inv) + (dbndiff2 * 2 * bndiff)

dbnmeani = (dbndiff * (- torch.ones_like(dbndiff))).sum(0, keepdim=True)

dhprebn = (dbndiff * torch.ones_like(bnmeani)) + (dbnmeani * (1/n) * torch.ones_like(hprebn))

dembcat = dhprebn @ W1.T

dW1 = embcat.T @ dhprebn

db1 = (dhprebn * torch.ones_like(embcat@W1)).sum(dim=0)

demb = dembcat.view(emb.shape)

dC = torch.zeros_like(C)

for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k][j]
        dC[ix] += demb[k][j]

# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bn_gain)
cmp('bnbias', dbnbias, bn_bias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)


logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: False | approximate: True  | maxdiff: 2.384185791015625e-07
counts_sum_inv  | exact: False | approximate: True  | maxdiff: 5.960464477539063e-08
counts_sum      | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
counts          | exact: False | approximate: True  | maxdiff: 2.9802322387695312e-08
norm_logits     | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
logit_maxes     | exact: False | approximate: True  | maxdiff: 6.984919309616089e-09
logits          | exact: False | approximate: True  | maxdiff: 6.51925802230835e-09
h               | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
W2              | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
b2              | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09
hpreact         | exact: False | approximate: True  | maxdiff: 1.862645149230957e

In [16]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.4894933700561523 diff: 2.384185791015625e-07


In [17]:
# backward pass

# -----------------

dlogits = F.softmax(logits, dim=1)
dlogits[range(n), Yb] -= 1
dlogits /= n

# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 1.0244548320770264e-08


In [18]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bn_gain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bn_bias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [19]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = bn_gain * bnvar_inv/n * (n * dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 1.862645149230957e-09


In [39]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_emb_dim = 10    # embedding dimensionality
n_hidden = 64    # neurons in hidden layers

g = torch.Generator().manual_seed(1337101)

C = torch.randn((vocab_size, n_emb_dim), generator=g)
# Layer 1
W1 = torch.randn((context_length*n_emb_dim, n_hidden), generator=g) * (5/3)/((context_length*n_emb_dim)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 # useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# batch normalization params
bn_gain = torch.ones((1, n_hidden))*0.1 + 1.0
bn_bias = torch.zeros((1, n_hidden))*0.1

# grads not required for these
# bn_mean_running = torch.zeros((1, n_hidden))
# bn_std_running = torch.ones((1, n_hidden))

# added bn_gain and bn_bias
parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]

for p in parameters:
    p.requires_grad = True

print(f'Total Parameters: {sum(p.nelement() for p in parameters)}')

# same optimization as last time

max_steps = 200000
batch_size = 24
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

# kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(-1, W1.shape[0]) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bn_gain * bnraw + bn_bias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    # loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    # YOUR CODE HERE :)

    # Cross Entropy
    dlogits = F.softmax(logits, dim=1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n

    # 2nd layer
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = (dlogits * torch.ones_like(h@W2)).sum(dim=0)

    # tanh
    dhpreact = dh * (1 - h**2)

    # BN
    dbngain = (dhpreact * bnraw).sum(dim=0, keepdim=True)
    dbnbias = (dhpreact * torch.ones_like(bn_bias)).sum(dim=0, keepdim=True)
    dhprebn = bn_gain * bnvar_inv/n * (n * dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = (dhprebn * torch.ones_like(embcat@W1)).sum(dim=0)

    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
        for j in range(Xb.shape[1]):
            ix = Xb[k, j]
            dC[ix] += demb[k, j]  

    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    # if i >= 150: # TODO: delete early breaking when you're ready to train the full net
    #   break

Total Parameters: 4137
      0/ 200000: 3.4895
  10000/ 200000: 2.1453
  20000/ 200000: 1.8336
  30000/ 200000: 1.9138
  40000/ 200000: 1.9173
  50000/ 200000: 1.8408
  60000/ 200000: 2.0505
  70000/ 200000: 1.9379
  80000/ 200000: 2.1220
  90000/ 200000: 2.0341
 100000/ 200000: 1.9613
 110000/ 200000: 1.8950
 120000/ 200000: 1.9836
 130000/ 200000: 2.1421
 140000/ 200000: 2.3674
 150000/ 200000: 2.0836
 160000/ 200000: 1.7635
 170000/ 200000: 1.9784
 180000/ 200000: 2.1936
 190000/ 200000: 2.1668


In [38]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

(27, 10)        | exact: False | approximate: True  | maxdiff: 5.122274160385132e-09
(30, 64)        | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(64,)           | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09
(64, 27)        | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(27,)           | exact: False | approximate: True  | maxdiff: 1.4901161193847656e-08
(1, 64)         | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
(1, 64)         | exact: False | approximate: True  | maxdiff: 7.450580596923828e-09


In [40]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [41]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bn_gain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bn_bias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 1.958603024482727
val 1.9605709314346313


In [42]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * context_length # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bn_gain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bn_bias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

chamahela.
jilvi.
kimraghivalankantha.
jananthan.
divathigeei.
neethishchaihvin.
neen.
dham.
powa.
qugan.
sulinth.
vidhi.
rajeep.
dearuxita.
jeepinsan.
sadevi.
abetha.
hasri.
aarasmithanksya.
pumara.
